In [1]:
# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

- [09 PyTorch 벤치마크](https://tutorials.pytorch.kr/recipes/recipes/benchmark.html)
- PyTorch benchmark모듈을 사용하여 코드 성능을 측정하고 비교하는 빠른 시작 가이드를 제공할 것임

# 소개
- 벤치마킹은 코드 작성에서 중요한 단계임
- 코드가 성능 기대치를 충족하는지 검증하고 동일한 문제를 해결하기 위한 다양한 접근 방식을 비교하며 성능 회귀를 방지하는 데 도움이 됨


- `timeit` Python 내장 모듈을 포함하여 PyTorch 코드를 벤치마킹할 때 많은 옵션이 있음
- 그러나 PyTorch 코드 벤치마킹에는 스레드 수 관리 및 CUDA 장치 동기화와 같이 쉽게 간과될 수 있는 많은 주의 사항이 있음
- 게다가 벤치마킹을 위해 Tensor 입력을 생성하는 것은 꽤 지루할 수 있음


- 이 레시피는 PyTorch `benchmark`모듈을 사용하여 일반적인 실수를 피하는 동시에 다른 코드의 성능을 더 쉽게 비교하고 벤치마킹을 위한 입력을 생성하는 방법을 보여줌

# 단계
1. 벤치마킹할 함수 정의
2. 벤치마킹 `timeit.Timer`
3. 벤치마킹 `torch.utils.benchmark.Timer`
4. 차단된 자동 범위를 사용한 벤치마킹
5. 벤치마크 결과 비교
6. 벤치마크 결과 저장/불러오기
7. 퍼지 매개변수 로 입력 생성
8. Callgrind로 명령어 카운트 수집

## 벤치마킹할 함수 정의
- 이 글을 쓰는 시점에서 `torch.dot`은 일괄 처리 모드를 지원하지 않으므로 기존 `torch` 연산자를 사용하여 구현하는 두 가지 접근 방식을 비교할 것임
- 하나는 `mul`과 `sum`의 조합을 사용하고, 다른 하나는 `bmm`을 사용


- cf)
- `mm` : matrix multiplication으로, [n, m] x [m,p] = [n,p] 를 구현함
- `bmm` : batch matrix multiplication으로, 두 operand가 모두 batch일 때 사용함
    - [B, n, m] x [B, m, p] = [B, n, p]

In [2]:
import torch

def batched_dot_mul_sum(a, b):
    '''Computes batched dot by multiplying and summing'''
    return a.mul(b).sum(-1)

def batched_dot_bmm(a, b):
    '''Computes batched dot by reducing to bmm'''
    a = a.reshape(-1, 1, a.shape[-1])
    b = b.reshape(-1, b.shape[-1], 1)
    return torch.bmm(a, b).flatten(-3)

# Input for benchmarking
x = torch.randn(10000, 64)

# Ensure that both functions compute the same output
assert batched_dot_mul_sum(x, x).allclose(batched_dot_bmm(x, x))

## 벤치마킹 `timeit.Timer`
- 먼저 Python의 내장 `timeit` 모듈을 사용하여 코드를 벤치마킹 해볼 것임
- 여기에서 벤치마크 코드를 단순하게 유지하여 `timeit`과 `torch.utils.benchmark`의 기본값을 비교할 수 있음

In [3]:
import timeit

t0 = timeit.Timer(
    stmt = 'batched_dot_mul_sum(x, x)',
    setup = 'from __main__ import batched_dot_mul_sum',
    globals={'x': x})

t1 = timeit.Timer(
    stmt = 'batched_dot_bmm(x, x)',
    setup = 'from __main__ import batched_dot_bmm',
    globals={'x': x})

print(f'mul_sum(x, x): {t0.timeit(100) / 100 * 1e6:>5.1f} us')
print(f'bmm(x, x): {t1.timeit(100) / 100 * 1e6:>5.1f} us')

mul_sum(x, x): 202.7 us
bmm(x, x): 262.0 us


# 벤치마킹 `torch.utils.benchmark.Timer`
- PyTorch `benchmark` 모듈은 이전에 `timeit` 모듈을 사용한 적이 있는 사람들에게 익숙하도록 설계됨
- 그러나 기본값들은 PyTorch 코드를 벤치마킹하는 것을 더 쉽고 안전하게 해줌
- 먼저 위와 동일한 기본 API를 비교해볼 것임

In [4]:
import torch.utils.benchmark as benchmark

t0 = benchmark.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='from __main__ import batched_dot_mul_sum',
    globals={'x': x})

t1 = benchmark.Timer(
    stmt='batched_dot_bmm(x, x)',
    setup='from __main__ import batched_dot_bmm',
    globals={'x': x})

print(t0.timeit(100))
print(t1.timeit(100))

batched_dot_mul_sum(x, x)
setup: from __main__ import batched_dot_mul_sum
  126.32 us
  1 measurement, 100 runs , 1 thread
batched_dot_bmm(x, x)
setup: from __main__ import batched_dot_bmm
  256.96 us
  1 measurement, 100 runs , 1 thread


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


- API는 기본 기능에 대해 동일하지만 몇 가지 중요한 차이점이 있음
- `benchmark.Timer.timeit()`은 총 런타임과 달리 실행당 시간을 반환함
- PyTorch `benchmark` 모듈은 결과 인쇄를 위한 형식화된 문자열 표현도 제공함


- 또 다른 중요한 차이점, 즉 결과가 다른 이유는 PyTorch 벤치마크 모듈이 기본적으로 단일 스레드에서 실행되기 때문임
- num_threads 인수로 스레드 수를 변경할 수 있음


- `torch.utils.benchmark.Timer`은 label, sub_label, description 및 env를 포함한 몇 가지 추가 인수를 취함
- 이 인수는 반환된 측정 개체의 `_repr__`를 변경하고 결과를 그룹화하는 데 사용됨(나중에 자세히 설명)

In [5]:
num_threads = torch.get_num_threads()
print(f'Benchmarking on {num_threads} threads')

t0 = benchmark.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='from __main__ import batched_dot_mul_sum',
    globals={'x': x},
    num_threads=num_threads,
    label='Multithreaded batch dot',
    sub_label='Implemented using mul and sum')

t1 = benchmark.Timer(
    stmt='batched_dot_bmm(x, x)',
    setup='from __main__ import batched_dot_bmm',
    globals={'x': x},
    num_threads=num_threads,
    label='Multithreaded batch dot',
    sub_label='Implemented using bmm')

print(t0.timeit(100))
print(t1.timeit(100))

Benchmarking on 3 threads
Multithreaded batch dot: Implemented using mul and sum
setup: from __main__ import batched_dot_mul_sum
  171.79 us
  1 measurement, 100 runs , 3 threads
Multithreaded batch dot: Implemented using bmm
setup: from __main__ import batched_dot_bmm
  255.45 us
  1 measurement, 100 runs , 3 threads


In [ ]:
# 일단 보류...